# Stoneburner, Kurt
- ## DSC 650 - Assignment 12


Importing Pillow images into Numpy
https://www.pluralsight.com/guides/importing-image-data-into-numpy-arrays

Pillow Resize Images: https://pythonexamples.org/python-pillow-resize-image/

Celeb A Dataset: http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html

In [1]:
import os
import sys
# //*** Imports and Load Data
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import time

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

#//*** Use the whole window in the IPYNB editor
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#//*** Clear the output display, helpful for content that updates
from IPython.display import clear_output


#//*** Maximize columns and rows displayed by pandas
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

import PIL
from PIL import Image
#//*** Get Working Directory
current_dir = Path(os.getcwd()).absolute()

#//*** Build Image Directory
base_image_dir = current_dir.parents[2].joinpath("img_align_celeba")

resize_dir = base_image_dir.joinpath("resize_28")

if os.path.exists(resize_dir) == False:
    os.mkdir( resize_dir)

ModuleNotFoundError: No module named 'tensorflow'

Tensorflow Datasets: https://www.tensorflow.org/datasets/overview

In [ ]:

#//**** 128x128 attributes
attribs = {
    'size' : (128,128),
    'keras_dims' : (128,128,3),

    'sample_size' : 1005,
    'test_split' : .5
}

#//**** 28x28 attributes
attribs = {
    'size' : (28,28),
    'keras_dims' : (28,28,3),

    'sample_size' : 100000,
    'test_split' : .1
}




x_train = []




counter = 1
for filename in os.listdir(base_image_dir)[:attribs['sample_size']]:
    clear_output(wait=True)
    
    
    #//*** Check if File has already been resized
    if os.path.exists(resize_dir.joinpath(filename)) == False:
        print("Resizing: ", counter,"/",attribs['sample_size'])

        #//*** File does not exist. Open original and resize
    
        #//*** Open Pillow Image Object
        image = Image.open(base_image_dir.joinpath(filename))
    
        #//*** Resize Image
        image = image.resize(attribs['size'])
        
        #//*** Save Image
        image.save(resize_dir.joinpath(filename))
         
        #//*** Add image to training set
        x_train.append( np.asarray(image) )
    
    else:
        print("Processing: ", counter,"/",attribs['sample_size'])
        #//*** File Exists, Load it
        #//*** Open Pillow Image Object
        x_train.append(np.asarray(Image.open(resize_dir.joinpath(filename))))
        
    counter += 1

    
    #//*** Display Images
    #plt.imshow(image.resize(attribs['dims']))
    #plt.show()
    #print(type(data))
    #print(data.shape)

#//*** Display a sample image
plt.imshow(Image.fromarray(x_train[0])   )
plt.show()

print(x_train[0].shape)

#//*** Index to split for test train
split_index = len(x_train) - int(len(x_train)*attribs['test_split'])
print(split_index)

x_test = x_train[split_index:]

x_train = x_train[:split_index]

print(len(x_train),len(x_test))

**Variational Autoencoder network**

In [ ]:
latent_dim = 3

encoder_inputs = keras.Input(shape=(attribs['keras_dims']))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var], name="encoder")

encoder.summary()

**Latent-Sapce-Sampling Layer**

In [ ]:
class Sampler(layers.Layer):
    def call(self, z_mean, z_log_var):
        batch_size = tf.shape(z_mean)[0]
        z_size = tf.shape(z_mean)[1]
        epsilon = tf.random.normal(shape=(batch_size, z_size))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon



**Variational decoder network, mapping latent space points to images**

In [ ]:
#//*** Dimensional Value. Convenience value for resizing the output layer to match the input size.
dv = 7
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(dv * dv * 64, activation="relu")(latent_inputs)
x = layers.Reshape((dv, dv, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2D(3, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")

decoder.summary()

**Variational Autoencoder with custom train_step()**

In [ ]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.sampler = Sampler()
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss")
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [self.total_loss_tracker,
                self.reconstruction_loss_tracker,
                self.kl_loss_tracker]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var = self.encoder(data)
            z = self.sampler(z_mean, z_log_var)
            reconstruction = decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction),
                    axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            total_loss = reconstruction_loss + tf.reduce_mean(kl_loss)
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "total_loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

**Training the Variational Auto Encoder**

In [ ]:
mnist_digits = np.concatenate([x_train, x_test], axis=0).astype("float32") / 255
#mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255



print(mnist_digits.shape)
#print(mnist_digits[0])

for epoch in range(20):
    
    vae = VAE(encoder, decoder)
    vae.compile(optimizer=keras.optimizers.Adam(), run_eagerly=True)
    vae.fit(mnist_digits, epochs=1, verbose=1, batch_size=128)


    import matplotlib.pyplot as plt
    
    
    if epoch  == 0 or (epoch+1)%5 == 0:
        print("==============")
        print("Epochs:",epoch+1)
        print("==============")
        

        n = 10
        digit_size = 128
        figure = np.zeros((digit_size * n, digit_size * n))

        grid_x = np.linspace(-1, 1, n)
        grid_y = np.linspace(-1, 1, n)[::-1]
        plt.figure(figsize=(40,40))
        count=1
        for i, yi in enumerate(grid_y):
            for j, xi in enumerate(grid_x):
                z_sample = np.array([[xi, yi,yi]])
                x_decoded = vae.decoder.predict(z_sample)[0]
                raw_img = (x_decoded*255).astype(np.uint8)
                plt.subplot(n,n,count)
                plt.imshow(Image.fromarray(raw_img)   )
                plt.axis("off")


                count+=1

        plt.show()
        
if (epoch+1)%5 != 0:
    print("==============")
    print("Epochs:",epoch+1)
    print("==============")


    n = 10
    digit_size = 128
    figure = np.zeros((digit_size * n, digit_size * n))

    grid_x = np.linspace(-1, 1, n)
    grid_y = np.linspace(-1, 1, n)[::-1]
    plt.figure(figsize=(40,40))
    count=1
    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z_sample = np.array([[xi, yi,yi]])
            x_decoded = vae.decoder.predict(z_sample)[0]
            raw_img = (x_decoded*255).astype(np.uint8)
            plt.subplot(n,n,count)
            plt.imshow(Image.fromarray(raw_img)   )
            plt.axis("off")


            count+=1

    plt.show()

**Sampling a grid of images from the 2D latent space**

In [ ]:
# //*** CODE HERE